In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import h5py
import imageio
import matplotlib
import gc
import sys
import io
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from scipy.optimize import curve_fit
import scipy.integrate

import scipy.ndimage

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(rf'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM
plt.rcParams.update({'font.size': 22})  

tidy3dAPI = os.environ["API_TIDY3D_KEY"]
plt.rc('font', family='Arial')


In [ ]:
folder_experimental = rf"H:\phd stuff\tidy3d\output\Experimental Data\data_lsu_air_decaline"
transmission_experimental_air = []
transmission_experimental_decaline = []

for filename in os.listdir(folder_experimental):
    if Path(filename).suffix == ".txt":
        continue
    with open(os.path.join(folder_experimental,filename), 'rb') as file:
        content = file.read()
        
    # Decode the binary content to a string
    decoded_content = content.decode('utf-8')

    # Split the content into lines
    lines = decoded_content.split('\r\n')

    # Initialize an empty list to store the parsed values
    data = []

    # Loop through each line and split by the comma
    for line in lines:
        if line:  # Skip any empty lines
            value1, value2 = line.split(',')
            data.append([float(value1), float(value2)])  # Convert values to float

    if "decaline" in filename:
        transmission_experimental_decaline.append((data))
    else: 
        transmission_experimental_air.append(data)


transmission_experimental_air = np.array(transmission_experimental_air)
transmission_experimental_decaline = np.array(transmission_experimental_decaline)

    

In [ ]:
sizes_exp = np.array([5.4,8.4,9.4,10.4,11.4,12.4,14.5,20.5])

In [ ]:
##Load simulation data 
file_sim = "20250627_far_field_eps_10p89_ff_20p73.h5"
sim_data = {}
with h5py.File(file_sim, 'r') as f:
    for item in f.keys():
        sim_data[item] = f[item][:]

file_sim_params = "20250627_far_field_eps_10p89_ff_20p73_params.h5"

sim_data_params = {}

with h5py.File(file_sim_params, 'r') as f:
    for item in f.keys():
        sim_data_params[item] = f[item][:]


In [ ]:
#Get parameters and reference 
r_proj=(sim_data_params["r_proj"])[0]
theta_proj=sim_data_params["theta_proj"]
phi_proj=sim_data_params["phi_proj"]
indices_15_30 = np.where(np.bool_(theta_proj*180/np.pi >=15) & np.bool_(theta_proj*180/np.pi <=30))[0]
r,theta,phi = r_proj,theta_proj[indices_15_30],phi_proj
lambdas = sim_data_params["lambdas"]
indices_degrees = np.where(np.bool_(theta_proj*180/np.pi >=15) & np.bool_(theta_proj*180/np.pi <=30))[0]

theta_slice = theta_proj[indices_degrees]
dtheta = np.diff(theta_slice)
dphi=np.diff(phi)[1]
theta_mid = 0.5 * (theta_slice[:-1] + theta_slice[1:]) 

dA = (r ** 2) * np.sin(theta_slice[1:]) * dtheta * dphi

transmission_15_30_degrees_0 = ((sim_data["reference.txt"][indices_degrees[:-1],:,:])* dA[:,np.newaxis,np.newaxis]).sum(axis=(0,1)) #Reference



In [ ]:
import re
shrikage = 0.95
plt.rcParams.update({'font.size': 15})  
fig, ax = plt.subplots(3,figsize=(10, 20),dpi=100)
for i,item in enumerate(sim_data.keys()):
    if item=="reference.txt":
        continue
    transmission_normalized_15_degrees = ((sim_data[item][indices_degrees[:-1],:,:])* dA[:,np.newaxis,np.newaxis]).sum(axis=(0,1))/transmission_15_30_degrees_0 
    sigma_tilted =14
    smoothed_transmission_15_degrees = scipy.ndimage.gaussian_filter1d(transmission_normalized_15_degrees, sigma_tilted)
    sigma_physical = np.diff(td.C_0/lambdas)[0] * sigma_tilted
    sigma_physical
    data_for_FS = [lambdas*shrikage]

    ####Get size 
    slab_size = 11.44*float(re.search(r'size_([+-]?\d+(?:\.\d+)?)', item).group(1))
    idx_eperiment = (np.abs(sizes_exp - slab_size)).argmin()
    ############

    ax[i].plot(shrikage*lambdas,smoothed_transmission_15_degrees,"*",label=rf"Convolution simulation")
    ax[i].plot(shrikage*lambdas,transmission_normalized_15_degrees,"o",label=rf"Simulation Cassegrain")
    data_for_FS.append(transmission_normalized_15_degrees)
    data_for_FS.append(smoothed_transmission_15_degrees)
    np.savetxt(rf"sim_transmision_n_3p3_ff_20_s_0p95_size_{slab_size:.2f}_um.txt", np.transpose(data_for_FS) , fmt="%.18f", delimiter=" ", header="lambda transmission_cassegrain convolution", comments="")
    np.savetxt(rf"experiment_transmision_n_3p3_ff_20_s_0p95_size_{slab_size:.2f}_um.txt", np.transpose([1e4/transmission_experimental_air[idx_eperiment,:,0],transmission_experimental_air[idx_eperiment,:,1]]) , fmt="%.18f", delimiter=" ", header="lambda transmission", comments="")

    ax[i].plot((1e4/transmission_experimental_air[idx_eperiment,:,0]),transmission_experimental_air[idx_eperiment,:,1],linewidth=4,label=rf"Experiment")
    ax[i].legend(loc=3)
    ax[i].grid(True, which="both")
    ax[i].set_yscale("log")
    ax[i].set_ylabel("Transmission")
    ax[i].set_title(rf"n_rod={3.3:.2f} $L={slab_size:.2f} \mu m$ ff_20% s={shrikage:.2f}")

    ax[i].set_xlim(np.min(lambdas*shrikage),np.max(lambdas*shrikage))
    ax[i].set_xlabel(rf'$\lambda (\mu m)$', fontsize=20)


plt.tight_layout()
plt.show()



In [33]:
np.median(td.C_0/freqs)/(td.C_0/sigma_physical)

0.058902536434362364